# Рекомендация тарифов

<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">
<b> Описание проекта</b>
    
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.
Постройте модель с максимально большим значением accuracy.


</div>

<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">
<b> Описание данных </b> 
 

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.

Известно:

сalls — количество звонков,

minutes — суммарная длительность звонков в минутах,

messages — количество sms-сообщений,

mb_used — израсходованный интернет-трафик в Мб,

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

</div>


 


## Открываем и изучаем файл

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('/users_behavior.csv')

<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">
Открыли файл, проверим содержимое:
</div>

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.sample(10, random_state=5)

,calls,minutes,messages,mb_used,is_ultra
3165,79.0,505.50,105.0,12406.00,0
426,36.0,198.96,32.0,21596.86,0
329,52.0,399.87,81.0,14055.95,1
232,42.0,260.94,79.0,17519.93,1
1525,58.0,403.84,2.0,13700.20,1
2440,62.0,466.60,0.0,17353.48,0
2033,78.0,398.25,82.0,17359.52,0
2366,37.0,263.57,0.0,18802.26,0
2226,10.0,69.79,14.0,31506.71,1
1435,76.0,585.56,30.0,22165.13,0


In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">
3214 записи со статистикой по абонентам, без пропусков. Нам прямо говорят, что предобработка данных не нужна, но я бы заменил типы данных 'calls' и 'messages' на целочисленные.

Переходм к следующему разделу.
</div>

## Разбиваем данные на выборки

<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">
Выделим из основног фрейма 25% строк на тестовую выборку, а из оставшихся данных выделим 25% строк на валидационную выборку, оставшиеся данные станут обучающей выборкой. Для этого используем функцию train_test_split из sklearn:
</div>

In [6]:
df_t_v, df_test = train_test_split(df, test_size=0.25, random_state=12345)
df_train, df_valid = train_test_split(df_t_v, test_size=0.25, random_state=12345)

In [7]:
print(' Размер Обучающей выборки:', df_train['is_ultra'].count(), '\n',
      'Размер Валидационной выборки:', df_valid['is_ultra'].count(), '\n',
      'Размер Тестовой выборки:', df_test['is_ultra'].count())

 Размер Обучающей выборки: 1807 
 Размер Валидационной выборки: 603 
 Размер Тестовой выборки: 804


<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">
Теперь разобъем каждую выборку на признаки и целевой признак 'is_ultra':
</div>

In [8]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

In [9]:
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

In [10]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">
Мы подготовили данные для обучения моделей и их исследования, переходим к следующему шагу.
</div>

## Обучаем и иследуем модели

### Дерево решений

<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">
Начнем с самой простой модели, а именно с Древа Решений. Для оптимизации пройдемся по уровням максимальной глубины дерева, обучим модель с глубиной от 1 до 10 и посмотрим показатель Accuracy для кажого из них. Помним, что при увеличении глубины Дерева решений мы увеличиваем склонность модели к переобучению, то есть к простому зазубриванию ответов, а не составлению модели предсказания. 
</div>

In [11]:
for i in range(1, 11):
    dtc_model = DecisionTreeClassifier(random_state = 12345, max_depth = i)
    dtc_model.fit(features_train, target_train)
    dtc_prediction = dtc_model.predict(features_valid)
    dtc_accuracy = accuracy_score(target_valid, dtc_prediction)
    print('max_depth =', i, ':', dtc_accuracy)

max_depth = 1 : 0.7495854063018242
max_depth = 2 : 0.7761194029850746
max_depth = 3 : 0.7943615257048093
max_depth = 4 : 0.7893864013266998
max_depth = 5 : 0.7877280265339967
max_depth = 6 : 0.7910447761194029
max_depth = 7 : 0.7827529021558872
max_depth = 8 : 0.7910447761194029
max_depth = 9 : 0.7744610281923715
max_depth = 10 : 0.7844112769485904


<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">
Показатель Accuracy максимален при глубине дерева равном 3-м уровням. Именно это значение гиперпараметра используем как основное на тестовой выборке позднее, но так же можно обратить внимание на уровни 6 и 8, как очень близкие по резельтатам.
</div>

### Случайный лес


<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">

Воторой моделью будет Случайный лес, логическое продолжение модели Дерева решений. Тут добавляется ещё один гипрер параметр - количество деревьев. Обучим модель на различном количестве деревьев и максимальных уровней глубины дерева:
    
    
- от 10 до 50 деревьев, с шагом 10;
    
- от 1 до 10 уровней глубины, с шагом 1.
    
Не станем выводить все 5 * 10 = 50 вариантов значения Accuracy, выведем только лучший.

</div>


In [12]:
rfc_best_model = None
rfc_best_accuracy = 0
rfc_best_est = 0
rfc_best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        rfc_model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est)
        rfc_model.fit(features_train, target_train)
        rfc_prediction = rfc_model.predict(features_valid)
        rfc_accuracy = accuracy_score(target_valid, rfc_prediction)
        if rfc_accuracy > rfc_best_accuracy:
            rfc_best_model = rfc_model
            rfc_best_accuracy = rfc_accuracy
            rfc_best_est = est
            rfc_best_depth = depth
print(" Accuracy наилучшей модели на валидационной выборке:", rfc_best_accuracy, '\n',
      "Количество деревьев:", rfc_best_est, '\n', "Максимальная глубина:", rfc_best_depth)            

 Accuracy наилучшей модели на валидационной выборке: 0.8374792703150912 
 Количество деревьев: 20 
 Максимальная глубина: 9


<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">
Показатель Accuracy максимален при количистве деревьев 20 и глубине дерева 9. Именно это значение гиперпараметров используем на тестовой выборке позднее.
</div>

### Логистическая регрессия


<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">

Третье моделью будет не склонная к переобучению Логистическая регрессия, которая не смотря на название выполняет классификацию, по весам признаков определяя к какому классу отнести каждый из наборов признаков.
</div>


In [13]:
lr_model = LogisticRegression(random_state=12345, solver='lbfgs')
lr_model.fit(features_train, target_train)
lr_result = lr_model.score(features_valid, target_valid)

print("Accuracy модели логистической регрессии на валидационной выборке:", lr_result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7412935323383084



<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">

Параметр 'solver' добавлен для устранения предупреждения, которое предостерегает нас от проблем наследования в будущих версиях этой модели.
    
А вот Accuracy на валидационной выборе ниже чем у предыдущих двух моделей, в дальнейшем сравним показатели на тестовой выборке.

</div>



<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">

Обучение и исследование завершили, переходим к проверке на тестовой выборке и выбору лучшей модели.

</div>


## Проверяем модель на тестовой выборке

### Дерево решений

<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">

Как решили выше к проверим модель на тестовой выборке не только с глубиной дерева равной 3-м, но и с близкими по Accuracy уровнями 6 и 8. Так же помним, что при проверке на валидационной выборе лидирует модель Случайного леса с максимальным уровнем 9. 
    
</div>

In [14]:
dtc_model = DecisionTreeClassifier(random_state = 12345, max_depth = 3)
dtc_model.fit(features_train, target_train)
dtc_prediction = dtc_model.predict(features_test)
dtc_accuracy = accuracy_score(target_test, dtc_prediction)
print('max_depth =', 3, ':', dtc_accuracy)

max_depth = 3 : 0.7910447761194029


In [15]:
dtc_model = DecisionTreeClassifier(random_state = 12345, max_depth = 6)
dtc_model.fit(features_train, target_train)
dtc_prediction = dtc_model.predict(features_test)
dtc_accuracy = accuracy_score(target_test, dtc_prediction)
print('max_depth =', 6, ':', dtc_accuracy)

max_depth = 6 : 0.7898009950248757


In [16]:
dtc_model = DecisionTreeClassifier(random_state = 12345, max_depth = 8)
dtc_model.fit(features_train, target_train)
dtc_prediction = dtc_model.predict(features_test)
dtc_accuracy = accuracy_score(target_test, dtc_prediction)
print('max_depth =', 8, ':', dtc_accuracy)

max_depth = 8 : 0.7922885572139303


<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">

Такой подход себя оправдал, видимо при большом количестве уровней переобучение модели не наступает, на тестовой выборке модель с параметром 8 показала даже лучший результат на 0.1%. Переходим к следующей модели.
    
</div>

### Случайный лес

<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">

Переходим к лидеру по Accuracy на валидационной выборке Случайному лесу, имея опыт из предыдущего шага, добавим ещё одну проверку модели с гиперпараметром максимальной глубины деревьев равной 8.
    
</div>

In [17]:
rfc_model = RandomForestClassifier(random_state=12345, max_depth=8, n_estimators=20)
rfc_model.fit(features_train, target_train)
rfc_prediction = rfc_model.predict(features_test)
rfc_accuracy = accuracy_score(target_test, rfc_prediction)
print(" Accuracy наилучшей модели на тестовой выборке:", rfc_accuracy, '\n',
      "Количество деревьев:", 20, '\n', "Максимальная глубина:", 8)

 Accuracy наилучшей модели на тестовой выборке: 0.7997512437810945 
 Количество деревьев: 20 
 Максимальная глубина: 8


In [18]:
rfc_model = RandomForestClassifier(random_state=12345, max_depth=9, n_estimators=20)
rfc_model.fit(features_train, target_train)
rfc_prediction = rfc_model.predict(features_test)
rfc_accuracy = accuracy_score(target_test, rfc_prediction)
print(" Accuracy наилучшей модели на тестовой выборке:", rfc_accuracy, '\n',
      "Количество деревьев:", 20, '\n', "Максимальная глубина:", 9)  

 Accuracy наилучшей модели на тестовой выборке: 0.8034825870646766 
 Количество деревьев: 20 
 Максимальная глубина: 9


<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">

Для начала, занчение гиперпараметра максимальной глудины полученной на валидационной выборке подтвердилось и на тестовой, показав лучшую точность.
    
А вот тончоность предсказания по сравнению на таестовой выборке по сравнению с валидационной снизилась, но в любом случае Accuracy более чем на 1% выше чем у модели Дерева решений.
    
Пока нашим лидером становится модель Случайного леса с количеством деревьев - 20 и максимальнйо глубиной дерева - 9.
    
</div>

### Логистическая регрессия

<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">

Переходим к показавшей самый скромный результат на валидаценной выборке Логистической регрессии. 
    
</div>

In [19]:
lr_model = LogisticRegression(random_state=12345, solver='lbfgs')
lr_model.fit(features_train, target_train)
lr_result = lr_model.score(features_test, target_test)

print("Accuracy модели логистической регрессии на тестовой выборке:", lr_result)

Accuracy модели логистической регрессии на тестовой выборке: 0.7611940298507462


<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">

Как и ранее Логистическая регрессия показала результат вписывающийся в обозначеный задачей минимум по точности, но уступает лидирующему Случайному лесу почти 5%.
    
</div>

### Вывод

<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">

Наше исследование показало, что предпочтительной моделью предсказания подходящего тарифа является Случайный лес с количеством деревьев равным 20 и максимальной глубиной деревьев раной 9.
    
Количество обучающих данных не вынуждает нас заботится о времени и ресурсах затраченных на обучение и исследовании моделей, но если такой вопрос встанет в дальнейшем, мы смело можем использовать модель Дерева предсказаний с максимальной глубиной дерева 8, мы выиграем в ресурсах и  с большой вероятностью не много потеряем в точности предсказаний.
    
А вот Логистическая регрессия не показала конкерентных результатов, в связи с чем отнесем эту модель к не подходящим к нашей задаче.
    
</div>

## (бонус) Проверьте модели на адекватность

<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">
    
Адекватность классификаций проверяется в сравнении с пропорционально случайно заполненными целевыми признаками, например если целевой признак имеет всего два класса и вероятность встретить каждый из классов в исторических данных равно 50%, то адекватной будет модель которая предсказывает значения целевого признака лучше чем случайность, а именно с показателем Accuracy более 0.50
    
Найдем на сколько часто встречаются наши классы (тарифы "Смарт" и  "Ультра") в фрейме:

</div>

In [20]:
df['is_ultra'].value_counts() / len(df)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

<div style="border-radius: 15px; border: 1px solid grey; padding: 15px;">
    
Получаем, что случйно заполенный целевой признак будет имеет соотношение ~(70\30), а все наши модели и, в первую очередь выбранная в финале, модель Случайного леса имеют точность предсказания выше 75%, таким образом все наши модели, проходившие проверку на тестовой выборке, можно признать адекватными.

</div>